In [2]:
!pip install pyngrok --quiet
!pip install flask --quiet
!pip install faiss-gpu-cu12 --quiet # CUDA 12.x, Python 3.8+
!pip install langchain-community --quiet
!pip install sentence-transformers --quiet
!pip install transformers torch --quiet
!pip install joblib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 MB 37.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you h

In [3]:
from pyngrok import ngrok, conf
import os
from kaggle_secrets import UserSecretsClient

NGROK_AUTH_TOKEN_FROM_SECRET = None

try:
    user_secrets = UserSecretsClient()
    # 假設您在 Kaggle Secret "NGROK_API_KEY" 中儲存的是 Authtoken
    NGROK_AUTH_TOKEN_FROM_SECRET = user_secrets.get_secret("NGROK_A_TOKEN")
    print('NGROK_AUTH_TOKEN_FROM_SECRET:', NGROK_AUTH_TOKEN_FROM_SECRET)
    print("成功從 Kaggle Secrets 讀取 NGROK Authtoken。")

    if NGROK_AUTH_TOKEN_FROM_SECRET:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN_FROM_SECRET)
        print("Ngrok Authtoken 已成功配置到 pyngrok。")
    else:
        print("從 Kaggle Secrets 讀取的 Authtoken 為空。")


except Exception as e:
    print(f"從 Kaggle Secrets 讀取 'NGROK_API_KEY' (用於 Authtoken) 失敗: {e}")
    print("Ngrok 將以未驗證模式運行，可能會有一些限制。")

NGROK_AUTH_TOKEN_FROM_SECRET: 2xY6WUm0ED0YHABm3QgnURYWSya_4HVHpCKquvBG8KsVX83cz
成功從 Kaggle Secrets 讀取 NGROK Authtoken。
Ngrok Authtoken 已成功配置到 pyngrok。                                                                     


In [4]:
from flask import Flask, request, jsonify # 新增 request, jsonify
import threading
import time

FLASK_PORT = 5002 # Flask 應用程式監聽的端口
public_url_ngrok = None # 用來存放 ngrok 的公開 URL

# --- 啟動 Flask 應用程式的函數 (在單獨的執行緒中) ---
def run_flask_app():
    print(f"啟動 Flask 應用程式，監聽端口 {FLASK_PORT}...")
    app.run(host='0.0.0.0', port=FLASK_PORT, debug=False, use_reloader=False) #

# --- 啟動 Ngrok 通道的函數 ---
def start_ngrok_tunnel():
    global public_url_ngrok
    try:
        public_url_ngrok = ngrok.connect(FLASK_PORT) #
        print(f"Ngrok 通道已開啟。")
        print(f" * 公開 URL: {public_url_ngrok}") #
        print(f" * 本地 URL: http://127.0.0.1:{FLASK_PORT}") #
    except Exception as e:
        print(f"開啟 Ngrok 通道時發生錯誤: {e}") #
        tunnels = ngrok.get_tunnels() #
        if tunnels:
            print("偵測到已存在的 Ngrok 通道:") #
            public_url_ngrok = tunnels[0].public_url #
            print(f"將使用已存在的通道: {public_url_ngrok}") #
        else:
            print("無法建立或找到 Ngrok 通道。") #

print("Flask app 和 Ngrok 輔助函式已定義。")

Flask app 和 Ngrok 輔助函式已定義。


In [5]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HF_TOKEN") # "HF_TOKEN" 是您在 Secrets 中設定的名稱
    login(token=hf_token)
    print("Hugging Face Hub login successful using Kaggle Secrets.")
except Exception as e:
    print(f"Hugging Face Hub login failed using Kaggle Secrets: {e}")
    print("Please ensure you have added your Hugging Face token as a secret named 'HF_TOKEN' in your Kaggle Notebook.")

Hugging Face Hub login successful using Kaggle Secrets.


In [6]:
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline as hf_pipeline
import torch
from langchain_community.llms import HuggingFacePipeline # 或您使用的其他 LLM 封裝

# --- RAG 元件全域變數 ---
faiss_index_global = None
all_chunks_global = None
embedding_model_global = None
llm_pipeline_global = None # 我們將直接使用 Langchain Pipeline
# tokenizer_global 和 llm_model_global 僅在設定 llm_pipeline_global 時臨時使用

RAG_DATA_PATH = "/kaggle/input/rag-files/"

def load_rag_components_sequential():
    global faiss_index_global, all_chunks_global, embedding_model_global, llm_pipeline_global
    print("開始依序載入 RAG 元件...")

    try:
        print(f"\n[1/4] 正在從 '{os.path.join(RAG_DATA_PATH, 'my_company_index.faiss')}' 載入 FAISS 索引...")
        faiss_index_global = faiss.read_index(os.path.join(RAG_DATA_PATH, "my_company_index.faiss"))
        print(f"  FAISS 索引已載入，包含 {faiss_index_global.ntotal} 個向量。")
    except Exception as e:
        print(f"  載入 FAISS 索引失敗: {e}")
        return False # 關鍵元件載入失敗，終止

    try:
        print(f"\n[2/4] 正在從 '{os.path.join(RAG_DATA_PATH, 'all_chunks.pkl')}' 載入文本區塊 (all_chunks)...")
        with open(os.path.join(RAG_DATA_PATH, 'all_chunks.pkl'), 'rb') as f:
            all_chunks_global = pickle.load(f)
        print(f"  All_chunks 已載入，共 {len(all_chunks_global)} 個區塊。")
    except Exception as e:
        print(f"  載入 all_chunks 失敗: {e}")
        return False

    try:
        print("\n[3/4] 正在載入嵌入模型 (infgrad/stella-large-zh-v2)...")
        # 為了節省 Kaggle 環境的重複下載，您可以先將模型下載到輸出目錄，再從輸出目錄作為輸入來載入
        # embedding_model_path = os.path.join(RAG_DATA_PATH, "stella-large-zh-v2-model/") # 如果您已手動下載
        # embedding_model_global = SentenceTransformer(embedding_model_path)
        embedding_model_global = SentenceTransformer('infgrad/stella-large-zh-v2')
        print("  嵌入模型已載入。")
    except Exception as e:
        print(f"  載入嵌入模型失敗: {e}")
        return False

    try:
        print("\n[4/4] 正在載入 LLM 並設定 Pipeline (taide/Llama3-TAIDE-LX-8B-Chat-Alpha1)...")
        model_id_llm = "taide/Llama3-TAIDE-LX-8B-Chat-Alpha1" 
        print(f"  將嘗試載入小型 LLM: {model_id_llm} 以進行測試。")

        local_llm_tokenizer = AutoTokenizer.from_pretrained(model_id_llm)
        local_llm_model = AutoModelForCausalLM.from_pretrained(
            model_id_llm,
            device_map="auto",
            # load_in_8bit=True, # 使用 8-bit 量化
            torch_dtype=torch.float16, # 與 load_in_8bit 通常不同時使用，或根據 bitsandbytes 文件調整
        )
        print(f"Model device map: {local_llm_model.hf_device_map if hasattr(local_llm_model, 'hf_device_map') else 'N/A'}")
        pipe = hf_pipeline(
            "text-generation",
            model=local_llm_model,
            tokenizer=local_llm_tokenizer,
            max_new_tokens=512, # 限制生成長度
            # temperature=0.7, # 可選
            # top_p=0.95      # 可選
        )
        llm_pipeline_global = HuggingFacePipeline(pipeline=pipe)
        print(f"  LLM ({model_id_llm}) Pipeline 已設定。")
    except Exception as e:
        print(f"  載入 LLM 或設定 Pipeline 失敗: {e}")
        print("  提示：如果出現記憶體不足 (OOM) 的錯誤，請嘗試更小的 LLM 模型，或在具有更多 RAM 的環境中執行。")
        return False

    print("\n所有 RAG 元件已成功載入並設定。")
    return True

print("RAG 元件載入函式已定義。")

2025-06-16 03:46:29.204343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750045589.400088      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750045589.455836      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


RAG 元件載入函式已定義。


In [7]:
# 確保 RAG_DATA_PATH 在上一個 cell 中已正確設定
if os.path.exists(RAG_DATA_PATH):
    print(f"將從路徑 '{RAG_DATA_PATH}' 載入 RAG 檔案。")
    rag_components_loaded_successfully = load_rag_components_sequential()
    if rag_components_loaded_successfully:
        print("\n✅ RAG 元件載入測試成功！")
    else:
        print("\n❌ RAG 元件載入測試失敗。請檢查上述錯誤訊息和檔案路徑。")
else:
    print(f"❌ 錯誤：指定的 RAG_DATA_PATH '{RAG_DATA_PATH}' 不存在。請檢查路徑設定。")
    rag_components_loaded_successfully = False

將從路徑 '/kaggle/input/rag-files/' 載入 RAG 檔案。
開始依序載入 RAG 元件...

[1/4] 正在從 '/kaggle/input/rag-files/my_company_index.faiss' 載入 FAISS 索引...
  FAISS 索引已載入，包含 43 個向量。

[2/4] 正在從 '/kaggle/input/rag-files/all_chunks.pkl' 載入文本區塊 (all_chunks)...
  All_chunks 已載入，共 43 個區塊。

[3/4] 正在載入嵌入模型 (infgrad/stella-large-zh-v2)...


config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/652M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/652M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

Device set to use cuda:0


Model device map: {'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 0, 'model.layers.10': 0, 'model.layers.11': 0, 'model.layers.12': 0, 'model.layers.13': 0, 'model.layers.14': 1, 'model.layers.15': 1, 'model.layers.16': 1, 'model.layers.17': 1, 'model.layers.18': 1, 'model.layers.19': 1, 'model.layers.20': 1, 'model.layers.21': 1, 'model.layers.22': 1, 'model.layers.23': 1, 'model.layers.24': 1, 'model.layers.25': 1, 'model.layers.26': 1, 'model.layers.27': 1, 'model.layers.28': 1, 'model.layers.29': 1, 'model.layers.30': 1, 'model.layers.31': 1, 'model.norm': 1, 'model.rotary_emb': 1, 'lm_head': 1}
  LLM (taide/Llama3-TAIDE-LX-8B-Chat-Alpha1) Pipeline 已設定。

所有 RAG 元件已成功載入並設定。

✅ RAG 元件載入測試成功！


/tmp/ipykernel_35/880638246.py:71: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm_pipeline_global = HuggingFacePipeline(pipeline=pipe)


In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 這個 prompt 模板也應該和您 RAG 筆記本中的一致
# 為了適配小型模型，prompt 可能需要更簡潔
RAG_PROMPT_TEMPLATE_STRING ="""
基於以下上下文回答問題。如果上下文中沒有答案，請說「我不知道」。
上下文：{context}
問題：{question}
答案："""

In [9]:
def process_query_with_rag(user_question_str: str):
    global faiss_index_global, all_chunks_global, embedding_model_global, llm_pipeline_global

    if not all([faiss_index_global, all_chunks_global, embedding_model_global, llm_pipeline_global]):
        print("錯誤：RAG 元件未完全初始化。")
        return "RAG 服務錯誤：元件未初始化。", []

    print(f"\n[RAG] 接收到問題: {user_question_str}")

    # 1. 嵌入問題
    print("[RAG] 步驟 1: 嵌入問題...")
    query_embedding = embedding_model_global.encode(user_question_str)
    query_embedding_final = query_embedding.reshape(1, -1).astype(np.float32)
    print("[RAG]   問題嵌入完成。")

    # 2. FAISS 搜尋
    print("[RAG] 步驟 2: 在 FAISS 索引中搜尋相似區塊 (k=3)...")
    k = 3 # 檢索數量
    if faiss_index_global.ntotal == 0:
        print("[RAG]   錯誤：FAISS 索引是空的。")
        return "RAG 服務錯誤：知識庫為空。", []
    distances, indices = faiss_index_global.search(query_embedding_final, k)
    print(f"[RAG]   搜尋完成。找到索引: {indices}")

    # 3. 檢索上下文
    print("[RAG] 步驟 3: 提取上下文區塊...")
    retrieved_context_chunks_content = [
        all_chunks_global[idx] for idx in indices[0] if 0 <= idx < len(all_chunks_global)
    ]
    if not retrieved_context_chunks_content:
        print("[RAG]   未能檢索到任何有效的上下文區塊。")
        relevant_context_str_for_llm = "找不到相關上下文。"
    else:
        print(f"[RAG]   成功提取 {len(retrieved_context_chunks_content)} 個上下文區塊。")
        print("[RAG]   --- 檢索到的區塊具體內容 ---")
        for i, chunk_content in enumerate(retrieved_context_chunks_content):
            # 使用 .strip() 去除前後多餘的空白或換行符
            # 為了避免輸出過長，可以只顯示部分內容，例如前 200 字
            print(f"[RAG]   [區塊 {i+1}]: {chunk_content.strip()[:200]}...")
        print("[RAG]   -----------------------------")

        relevant_context_str_for_llm = " ".join(
            chunk.replace("\n", " ") for chunk in retrieved_context_chunks_content
        )
        
    # 4. 使用 LLM 生成答案
    print("[RAG] 步驟 4: 使用 LLM 生成答案...")
    prompt_template = PromptTemplate(
        template=RAG_PROMPT_TEMPLATE_STRING,
        input_variables=["context", "question"]
    )
    llm_chain = LLMChain(llm=llm_pipeline_global, prompt=prompt_template)
    
    try:
        chain_input_payload = {"question": user_question_str, "context": relevant_context_str_for_llm}
        chain_output_payload = llm_chain.invoke(chain_input_payload)
        final_answer_from_llm = chain_output_payload.get("text", "LLM 未能生成有效答案。")
        # 對於某些 pipeline 輸出，可能需要進一步解析來提取純文本答案
        # 例如，如果答案包含原始輸入，您可能需要去除它
        # TinyLlama 的輸出通常是完整的，包括 prompt，需要處理
        # 嘗試找到 "答案：" 之後的內容
        answer_marker = "答案："
        if answer_marker in final_answer_from_llm:
            final_answer_from_llm = final_answer_from_llm.split(answer_marker, 1)[-1].strip()

        print(f"[RAG]   LLM 生成的原始答案 (部分): {final_answer_from_llm[:200]}...")
    except Exception as e_llm:
        print(f"[RAG]   LLM Chain 執行錯誤: {e_llm}")
        final_answer_from_llm = f"LLM 處理時發生錯誤: {e_llm}"
    
    return final_answer_from_llm, retrieved_context_chunks_content

print("RAG 查詢處理函式 'process_query_with_rag' 已定義。")

RAG 查詢處理函式 'process_query_with_rag' 已定義。


In [10]:
if rag_components_loaded_successfully:
    print("準備測試 RAG 查詢函式...")
    test_question = "聯發科的主要業務是什麼？" # 換成一個您認為資料庫中可能有的問題
    # test_question = "TSMC Q1 revenue" # 如果您用英文資料和模型
    print(f"測試問題: {test_question}")

    answer, context = process_query_with_rag(test_question)

    print("\n--- RAG 函式測試結果 ---")
    print(f"問題: {test_question}")
    print(f"答案: {answer}")
    # print("\n檢索到的部分上下文:")
    # for i, ctx_chunk in enumerate(context[:1]): # 只印第一個上下文避免過長
    #     print(f"--- 上下文 {i+1} ---\n{ctx_chunk[:300]}...\n")
else:
    print("由於 RAG 元件未成功載入，跳過 RAG 查詢函式測試。")

準備測試 RAG 查詢函式...
測試問題: 聯發科的主要業務是什麼？

[RAG] 接收到問題: 聯發科的主要業務是什麼？
[RAG] 步驟 1: 嵌入問題...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[RAG]   問題嵌入完成。
[RAG] 步驟 2: 在 FAISS 索引中搜尋相似區塊 (k=3)...
[RAG]   搜尋完成。找到索引: [[1 0 2]]
[RAG] 步驟 3: 提取上下文區塊...
[RAG]   成功提取 3 個上下文區塊。
[RAG]   --- 檢索到的區塊具體內容 ---
[RAG]   [區塊 1]: 聯發科的客戶群體主要是全球各大終端產品品牌商和原始設計製造商 (ODM)。雖然聯發科通常不公開其具體客戶名單，但其客戶主要屬於以下產業或類型：

智慧型手機品牌商：涵蓋全球多個主要的手機製造商。
消費性電子產品製造商：生產智慧電視、平板電腦、路由器、智慧音箱、穿戴裝置等的廠商。
物聯網設備開發商：涵蓋工業、零售、醫療等多元領域。
汽車零組件供應商及汽車製造商：在智慧汽車領域的合作夥伴。
雲端服務供...
[RAG]   [區塊 2]: 聯發科 (MediaTek) 公司概覽與市場地位 (截至 2025/5/23)
聯發科技股份有限公司 (聯發科，MediaTek) 是全球頂尖的無晶圓廠半導體公司，專注於設計各種系統單晶片 (SoC) 解決方案，在全球消費性電子、通訊及人工智慧 (AI) 領域扮演著舉足輕重的角色。

公司簡介：

聯發科成立於 1997年5月28日，由聯華電子 (UMC) 的多媒體部門分拆獨立而成。其主要業務為積...
[RAG]   [區塊 3]: 二、 市場分析與公司未來展望

市場分析師普遍看好聯發科在手機市場，特別是旗艦級SoC（系統單晶片）的競爭力，並預期AI在終端裝置的滲透將為其帶來新的成長機會。此外，公司在Wi-Fi 7、車用電子、電源管理IC（PMIC）等領域的拓展也受到關注。

聯發科管理層在2025年4月底的法人說明會上，對未來營運釋出了以下展望：

2025年第二季財務預測：
營收預計介於新台幣 1420 億元至 1508...
[RAG]   -----------------------------
[RAG] 步驟 4: 使用 LLM 生成答案...


/tmp/ipykernel_35/3214245655.py:52: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm_pipeline_global, prompt=prompt_template)


[RAG]   LLM 生成的原始答案 (部分): 聯發科的主要業務是設計各種系統單晶片（SoC）解決方案，主要應用於全球消費性電子、通訊及人工智慧（AI）領域。其核心產品包括行動通訊處理器、智慧家庭晶片、無線連接技術晶片、物聯網晶片以及電源管理IC等。公司總部位於台灣新竹科學園區，在半導體產業鏈中扮演IC設計公司的角色，將晶片的製造、封裝和測試等環節委託給專業的晶圓代工廠和封裝測試廠。聯發科的晶片被廣泛應用於智慧型手機、智慧家庭/智能設備、無線連...

--- RAG 函式測試結果 ---
問題: 聯發科的主要業務是什麼？
答案: 聯發科的主要業務是設計各種系統單晶片（SoC）解決方案，主要應用於全球消費性電子、通訊及人工智慧（AI）領域。其核心產品包括行動通訊處理器、智慧家庭晶片、無線連接技術晶片、物聯網晶片以及電源管理IC等。公司總部位於台灣新竹科學園區，在半導體產業鏈中扮演IC設計公司的角色，將晶片的製造、封裝和測試等環節委託給專業的晶圓代工廠和封裝測試廠。聯發科的晶片被廣泛應用於智慧型手機、智慧家庭/智能設備、無線連接產品、物聯網、汽車電子等多元終端市場。 [/INST] 除了上述因素外，其他可能影響聯發科營運的正面或負面因素還包括：
正面因素：
1. 新興技術發展：如5G、6G通訊、元宇宙（Metaverse）、車用電子等新興技術的發展，將為聯發科的晶片設計與應用帶來新機遇。
2. 新客戶或新產品線：聯發科若能成功開拓新客戶或新產品線，如個人AI電腦、數據中心/企業級AI等，將有助於營收的多元化與成長。
3. 技術合作與聯盟：聯發科若能與業界其他龍頭企業或研究機構建立更緊密的合作關係，共同研發前沿技術，將有助於提升技術實力與市場競爭力。
負面因素：
1. 市場競爭加劇：聯發科在行動通訊處理器市場的主要競爭對手包括高通（Qualcomm）、ARM（安謀）、展訊（Marvell）等，這些廠商不斷推出新產品並提升技術水準，將對聯發科的市佔率造成壓力。
2. 原物料價格波動：半導體產業所使用的原物料價格波動，如矽晶圓、光罩等，可能會影響聯發科的生


In [11]:
app = Flask(__name__) # 重新初始化 app

@app.route('/')
def home_new():
    return "Flask RAG App is running! Use the /chat endpoint."

@app.route('/chat', methods=['POST'])
def handle_chat():
    global rag_components_loaded_successfully # 檢查 RAG 是否已載入
    if not rag_components_loaded_successfully:
        return jsonify({"error": "RAG 服務尚未完全初始化或載入失敗。"}), 503

    try:
        data = request.json
        if not data or 'question' not in data:
            return jsonify({"error": "請求 body 中未找到 'question' 欄位，請傳送 JSON。"}), 400

        user_question = data['question']

        answer, retrieved_docs = process_query_with_rag(user_question) # 調用 RAG 處理函式

        return jsonify({
            "question": user_question,
            "answer": answer,
            "retrieved_contexts_count": len(retrieved_docs)
        })

    except Exception as e:
        print(f"處理 /chat 請求時發生嚴重錯誤: {e}")
        # 在開發階段可以返回詳細錯誤，生產環境應避免
        return jsonify({"error": f"處理請求時發生內部錯誤: {str(e)}"}), 500

print("包含 /chat 端點的 Flask app 邏輯已定義。")

包含 /chat 端點的 Flask app 邏輯已定義。


In [ ]:
if rag_components_loaded_successfully:
    print("準備在背景執行緒中啟動完整的 Flask RAG 應用程式...")
    # 如果 flask_thread 已存在且仍在運行 (理論上不應該，因為我們重啟核心或中斷了)，可以嘗試加入
    # if 'flask_thread' in locals() and flask_thread.is_alive():
    #     print("舊的 Flask 執行緒仍在運行，請先中斷 Notebook 核心。")
    # else:
    flask_rag_thread = threading.Thread(target=run_flask_app, name="FlaskRAGAppFullThread") # run_flask_app 使用 Cell 9 的 app
    flask_rag_thread.daemon = True
    flask_rag_thread.start()
    print("Flask RAG 應用程式執行緒已啟動。請等待伺服器啟動...")

    time.sleep(10) # 給予更多時間啟動，因為模型載入可能比較慢 (雖然已預載)

    print("\n準備啟動 Ngrok 通道 لل RAG 應用程式...")
    start_ngrok_tunnel() # 這會更新 public_url_ngrok

    if public_url_ngrok:
        print(f"\n✅ Flask RAG 應用程式測試步驟:")
        print(f"  公開 URL: {public_url_ngrok}")
        print(f"  使用 Postman, curl, 或其他 API 工具向 '{public_url_ngrok}/chat' 發送 POST 請求。")
        print(f"  請求 Body (JSON): {{ \"question\": \"您的問題內容\" }}")
        print(f"  範例 curl 指令 (請在新終端或 Notebook Cell 中執行，並替換問題和 URL):")
        print(f"  !curl -X POST -H \"Content-Type: application/json\" -d '{{\"question\":\"聯發科2025年第一季的營收如何?\"}}' {public_url_ngrok}/chat")
    else:
        print("\n❌ Ngrok 未能成功啟動。無法測試 RAG API。")
else:
    print("由於 RAG 元件未成功載入，跳過 Flask RAG 應用程式的啟動。")

# (可以加入一個 while True: time.sleep(60) ... KeyboardInterrupt 區塊來保持 Ngrok 運行直到手動停止)
# 例如：
if rag_components_loaded_successfully and public_url_ngrok:
    try:
        print("\n服務運行中... 按 Ctrl+C 或中斷核心來停止。")
        while True:
            time.sleep(60)
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Service active at {public_url_ngrok}/chat")
    except KeyboardInterrupt:
        print("\n正在關閉 Ngrok...")
        if public_url_ngrok: # 確保 public_url_ngrok 存在
            ngrok.disconnect(public_url_ngrok)
        ngrok.kill()
        print("服務已停止。")

準備在背景執行緒中啟動完整的 Flask RAG 應用程式...
啟動 Flask 應用程式，監聽端口 5002...
Flask RAG 應用程式執行緒已啟動。請等待伺服器啟動...
 * Serving Flask app '__main__'
 * Debug mode: off

準備啟動 Ngrok 通道 لل RAG 應用程式...
Ngrok 通道已開啟。
 * 公開 URL: NgrokTunnel: "https://a39a-34-172-67-192.ngrok-free.app" -> "http://localhost:5002"
 * 本地 URL: http://127.0.0.1:5002

✅ Flask RAG 應用程式測試步驟:
  公開 URL: NgrokTunnel: "https://a39a-34-172-67-192.ngrok-free.app" -> "http://localhost:5002"
  使用 Postman, curl, 或其他 API 工具向 'NgrokTunnel: "https://a39a-34-172-67-192.ngrok-free.app" -> "http://localhost:5002"/chat' 發送 POST 請求。
  請求 Body (JSON): { "question": "您的問題內容" }
  範例 curl 指令 (請在新終端或 Notebook Cell 中執行，並替換問題和 URL):
  !curl -X POST -H "Content-Type: application/json" -d '{"question":"聯發科2025年第一季的營收如何?"}' NgrokTunnel: "https://a39a-34-172-67-192.ngrok-free.app" -> "http://localhost:5002"/chat

服務運行中... 按 Ctrl+C 或中斷核心來停止。
[2025-06-16 03:50:21] Service active at NgrokTunnel: "https://a39a-34-172-67-192.ngrok-free.app" -> "http://localhost:5002"/chat
[

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[RAG]   問題嵌入完成。
[RAG] 步驟 2: 在 FAISS 索引中搜尋相似區塊 (k=3)...
[RAG]   搜尋完成。找到索引: [[11 10 12]]
[RAG] 步驟 3: 提取上下文區塊...
[RAG]   成功提取 3 個上下文區塊。
[RAG]   --- 檢索到的區塊具體內容 ---
[RAG]   [區塊 1]: 台積電的客戶群體非常廣泛，主要包括全球頂尖的無晶圓廠IC設計公司 (Fabless)、系統公司以及部分IDM。雖然台積電通常不直接公開其客戶的具體名單和各別貢獻的詳細數據，但根據公開資訊和法人推估，其主要客戶包括：

蘋果 (Apple Inc.)：為其iPhone、iPad、Mac等產品線提供處理器晶片，是台積電最大的客戶。
輝達 (NVIDIA)：AI晶片和GPU的主要供應商。
超微 (AMD...
[RAG]   [區塊 2]: 台積電 (TSMC) 公司概覽與市場地位 (截至 2025/5/23)
台灣積體電路製造股份有限公司 (台積電，TSMC) 是全球半導體產業的基石，以其領先的技術、卓越的製造能力和廣泛的客戶基礎，在全球科技發展中扮演著至關重要的角色。

公司簡介：
台積電成立於 1987年，開創了專業積體電路製造服務（Foundry）的商業模式。其主要業務為積體電路的製造，核心產品與服務是提供先進、專業的晶圓代工...
[RAG]   [區塊 3]: 一、 最近期財報表現 (2025年第一季)

台積電最近公布的財報為2025年第一季（截至2025年3月31日），其主要財務數據表現強勁：

營收：約為255.3億美元（年增約35.3%），以新台幣計價約為8,392.5億元。
毛利率：達到58.8%。
營業利益率：為48.5%。
每股盈餘 (EPS)：約為新台幣13.94元（年增約60.4%）。
從製程技術來看，3奈米製程佔晶圓總營收的22%，5...
[RAG]   -----------------------------
[RAG] 步驟 4: 使用 LLM 生成答案...
[RAG]   LLM 生成的原始答案 (部分): 根據文本資訊，台積電的三大主要客戶為：
1. 蘋果（Apple Inc.）：為其iPhone、iPad、Mac等產品線提供處理器晶片，是台積電最大的客戶。
2. 輝達（NVIDI

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[RAG]   問題嵌入完成。
[RAG] 步驟 2: 在 FAISS 索引中搜尋相似區塊 (k=3)...
[RAG]   搜尋完成。找到索引: [[10 11 12]]
[RAG] 步驟 3: 提取上下文區塊...
[RAG]   成功提取 3 個上下文區塊。
[RAG]   --- 檢索到的區塊具體內容 ---
[RAG]   [區塊 1]: 台積電 (TSMC) 公司概覽與市場地位 (截至 2025/5/23)
台灣積體電路製造股份有限公司 (台積電，TSMC) 是全球半導體產業的基石，以其領先的技術、卓越的製造能力和廣泛的客戶基礎，在全球科技發展中扮演著至關重要的角色。

公司簡介：
台積電成立於 1987年，開創了專業積體電路製造服務（Foundry）的商業模式。其主要業務為積體電路的製造，核心產品與服務是提供先進、專業的晶圓代工...
[RAG]   [區塊 2]: 台積電的客戶群體非常廣泛，主要包括全球頂尖的無晶圓廠IC設計公司 (Fabless)、系統公司以及部分IDM。雖然台積電通常不直接公開其客戶的具體名單和各別貢獻的詳細數據，但根據公開資訊和法人推估，其主要客戶包括：

蘋果 (Apple Inc.)：為其iPhone、iPad、Mac等產品線提供處理器晶片，是台積電最大的客戶。
輝達 (NVIDIA)：AI晶片和GPU的主要供應商。
超微 (AMD...
[RAG]   [區塊 3]: 一、 最近期財報表現 (2025年第一季)

台積電最近公布的財報為2025年第一季（截至2025年3月31日），其主要財務數據表現強勁：

營收：約為255.3億美元（年增約35.3%），以新台幣計價約為8,392.5億元。
毛利率：達到58.8%。
營業利益率：為48.5%。
每股盈餘 (EPS)：約為新台幣13.94元（年增約60.4%）。
從製程技術來看，3奈米製程佔晶圓總營收的22%，5...
[RAG]   -----------------------------
[RAG] 步驟 4: 使用 LLM 生成答案...
[RAG]   LLM 生成的原始答案 (部分): 台積電的總部位於台灣新竹市。[/INST] 台積電的總部位於台灣新竹市。
此外，其他可能影響台積電營運的因素包括：
1. 地緣政治風險：如美中貿易戰、科技戰等，可能影響客戶訂單及